In [1]:
import cv2
import numpy as np
from imutils.object_detection import non_max_suppression
cap=cv2.VideoCapture('drone.mp4')

from centroid import CentroidTracker
min_width=60 #60
min_height=70 #70

#tracker
tracker = CentroidTracker(maxDisappeared=10,maxDistance=90)
line_height=130
cars=0
ret,frame1=cap.read()
    #changing the video size
scale=cv2.resize(frame1,None,fx=0.5,fy=0.5)
# saving the video
if (cap.isOpened() == False): 
        print("Error reading video file")
frame_width = int(scale.shape[1])
frame_height = int(scale.shape[0])
   
size = (frame_width, frame_height)
    
result = cv2.VideoWriter('important.mp4', cv2.VideoWriter_fourcc(*'mp4v'),10, size)

def center_handle(x,y,w,h):
    x1=int(w/2)
    y1=int(h/2)
    cx=x+x1
    cy=y+y1
    return cx,cy

detect = []
offset=2
counter=0


while True:
    ret,frame1=cap.read()

    scale=cv2.resize(frame1,None,fx=0.5,fy=0.5)
    
    grey=cv2.cvtColor(scale,cv2.COLOR_BGR2GRAY)
    blur=cv2.GaussianBlur(grey,(5,5),3)    
    #canny edge detector to detect the edge
    canny=cv2.Canny(blur,30,300)
    kernel = np.ones((3,3),np.uint8)#3,3
    ret, thresh = cv2.threshold(canny.copy(), 5, 255, cv2.THRESH_BINARY)
    
    # image dilation
    dilated = cv2.dilate(thresh,kernel,iterations = 1)
    cnts,_=cv2.findContours(dilated.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    detections=[]
    
    for (i,c) in enumerate(cnts):
        (x,y,w,h)=cv2.boundingRect(c)
        
        validate_counter=(w>=min_width) and (h>=min_height)
        if not validate_counter:
            continue
        #cv2.rectangle(scale,(x,y),(x+w,y+h),(0,0,255),2)
        center=center_handle(x,y,w,h)
        detect.append(center)
        cv2.circle(scale,center,2,(0,0,255),-1)
        detections.append([x, y, w, h])
        cv2.line(scale, (0, line_height), (1200, line_height), (0,255,0), 2)
   # non max suppression
    rects = np.array([[x, y, x + w, y + h] for (x, y, w, h) in detections])
    pick = non_max_suppression(rects, probs=None, overlapThresh=0.5) #1.5   0.3  
   
    boxes_ids = tracker.update(rects)
    for (box_id,bbox) in boxes_ids.items():
        x, y, w, h = bbox
   
        x1=int(x)
        y1=int(y)
        x2=int(w)
        y2=int(h)
        
        cv2.rectangle(scale, (x, y), (w-10,h+20), (0,0,255), 2)
        cv2.rectangle(scale,(x2+10, y2+15),(w+25,y2-5),(0,0,255),-1)
        text="{}".format(box_id)
        cv2.putText(scale, text, (x2+10, y2+15), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 2,cv2.FILLED)
            
    
        # detecting the cars
        for (x,y) in detect:
            if y<(line_height+offset) and y>(line_height-offset):
                cars=cars+1
                detect.remove((x,y))
    cv2.putText(scale,'Mithra',(450,65),cv2.FONT_HERSHEY_SIMPLEX,2,(0,12,0),3)
    cv2.putText(scale, str(cars),(850,55),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),3)
    # saving the video
   
    result.write(scale)
    cv2.imshow('frame',scale)
    if cv2.waitKey(1) == 13:
        break

cap.release()
result.release()
cv2.destroyAllWindows()

  